## Analiza danych
### Hubert Kłosowski 242424
### Kamil Małecki 242464

### Potrzebne importy

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import datetime as dt
from sklearn.preprocessing import PolynomialFeatures

### Wczytanie danych

In [ ]:
def get_data():
    os.chdir('..')
    os.chdir('Zadanie1\\czesc4')
    read = []
    for filename in os.listdir():
        df = pd.read_csv(filename)
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'])
        read.append(df)
    os.chdir('..')
    os.chdir('..')
    os.chdir('Zadanie2')
    return read

data = get_data()
part6 = data[5]

## <center>Część 1</center>
### Wyznacz średnią kroczącą i odchylenie standardowe kroczące (np. 7-dniowe), aby zobaczyć, jak te zjawiska zmieniają się w czasie. Obliczenia wykonaj dla:


### najbogatszego kraju z każdego kontynentu

In [ ]:
countries = part6[['country_name', 'gdp_usd', 'Continent']].sort_values(by='gdp_usd', ascending=False).drop_duplicates().groupby('Continent')
continents = part6['Continent'].unique()
choose_countries = np.array([countries.get_group(continent)['country_name'][:1].values for continent in continents])
choose_countries = choose_countries.reshape(-1)

### Średnia ruchoma i odchylenie standardowe

In [ ]:
def calculate_moving_avg_std(column, country_name):
    mean_data = part6.loc[part6['country_name'] == country_name, column].rolling(window=7).mean().bfill()
    std_data = part6.loc[part6['country_name'] == country_name, column].rolling(window=7).std().bfill()
    return pd.DataFrame({'mean': mean_data, 'std': std_data}).reset_index(drop=True)

def plot_moving_avg_std(ylabel, column, trend_line=False):
    fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(20, 12))
    fig.suptitle('Średnia i ochylenie standardowe kroczące 7-dniowe', fontsize=20)
    for i, country_name in enumerate(choose_countries):
        x_cord, y_cord = divmod(i, 2)
        moving_avg_std = calculate_moving_avg_std(column, country_name)
        x_axis = part6['date'].unique()
        sns.lineplot(y=moving_avg_std['mean'], x=x_axis, ax=ax[x_cord, y_cord], color='red', label='średnia krocząca')
        sns.lineplot(y=moving_avg_std['std'], x=x_axis, ax=ax[x_cord, y_cord], color='green', label='odchylenie standardowe kroczące')
        if trend_line:
            x = x_axis.map(dt.datetime.toordinal)
            mean_trend_line = np.poly1d(np.polyfit(x, moving_avg_std['mean'], 1))
            std_trend_line = np.poly1d(np.polyfit(x, moving_avg_std['std'], 1))
            sns.lineplot(y=mean_trend_line(x), x=x_axis, ax=ax[x_cord, y_cord], color='red', linestyle='dashed')
            sns.lineplot(y=std_trend_line(x), x=x_axis, ax=ax[x_cord, y_cord], color='green', linestyle='dashed')
        ax[x_cord, y_cord].set_title(country_name)
        ax[x_cord, y_cord].set_xlabel('')
        ax[x_cord, y_cord].set_ylabel('')
        if x_cord == 2:
            ax[x_cord, y_cord].set_xlabel('Date')
        if y_cord == 0:
            ax[x_cord, y_cord].set_ylabel(ylabel)
        ax[x_cord, y_cord].legend(loc='best')

### 1.1. liczby nowych zachorowań,

In [ ]:
plot_moving_avg_std('Liczba nowych zachorowań', 'daily_confirmed')

### 1.2. liczby nowych śmierci,

In [ ]:
plot_moving_avg_std('Liczba nowych śmierci', 'daily_deceased')

### 1.3. liczby nowych szczepień,

In [ ]:
plot_moving_avg_std('Liczba nowych szczepień', 'daily_persons_vaccinated')

### 1.4 liczba nowych zachorowań mężczyzn,

In [ ]:
plot_moving_avg_std('Liczba nowych zachorowań mężczyzn', 'daily_confirmed_male')

### 1.4 liczba nowych śmierci mężczyzn

In [ ]:
plot_moving_avg_std('Liczba nowych śmierci mężczyzn', 'daily_deceased_male')

## <center>Część 2</center>
### Wykorzystaj do analizy trendów analizę szeregów czasowych (metoda średniej ruchomej czy inne modele autoregresyjne), która umożliwi zbadanie 5 przypadków rozważanych w części 1 poziomu 3. Przeanalizuj otrzymane wyniki.

### 1.1 liczba nowych zachorowań,

In [ ]:
plot_moving_avg_std('Liczba nowych zachorowań', 'daily_confirmed', trend_line=True)

In [ ]:
pd.date_range(part6['date'].min(), part6['date'].max(), freq='W-WED')

### 1.2. liczby nowych śmierci,

In [ ]:
plot_moving_avg_std('Liczba nowych śmierci', 'daily_deceased', trend_line=True)

### 1.3. liczby nowych szczepień,

In [ ]:
plot_moving_avg_std('Liczba nowych szczepień', 'daily_persons_vaccinated', trend_line=True)

### 1.4 liczba nowych zachorowań mężczyzn,

In [ ]:
plot_moving_avg_std('Liczba nowych zachorowań mężczyzn', 'daily_confirmed_male', trend_line=True)

### 1.4 liczba nowych śmierci mężczyzn

In [ ]:
plot_moving_avg_std('Liczba nowych śmierci mężczyzn', 'daily_deceased_male', trend_line=True)

## <center>Cześć 3</center>
### Przygotuj dane treningowe w interesującym Cię okresie czasu (np. druga połowa 2020 roku i pierwsza połowa 2021 roku, tj. 52 tygodnie = X) oraz ewentualne dane testowe. Możesz przefiltrować dane także po innych kryteriach, jeżeli uznasz to za potrzebne. Zastosuj analizę regresji, aby przewidzieć wartości w kolejnych X tygodniach następujących po wybranym okresie treningowym. Użyj modelu regresji liniowej, gdzie zmienną niezależną będzie czas, a zmienną zależną:

### Dane treningowe i testowe

In [ ]:
def get_train_test(column, country, start, end, duration):
    train_data = part6.loc[(part6['country_name'] == country) & (part6['date'].between(start, end)), ['date', column]]
    test_data = part6.loc[(part6['country_name'] == country) & (part6['date'].between(pd.to_datetime(end) + pd.Timedelta(1, unit='d'), pd.to_datetime(end) + pd.Timedelta(1 + 7 * duration, unit='d'))), ['date', column]]
    return train_data, test_data

### Model regresji liniowej

In [ ]:
def linear_regression(train, test, ylabel, column, poly, ax=None):
    reg = LinearRegression(n_jobs=-1)
    map_train_dates = train['date'].map(dt.datetime.toordinal)
    map_test_dates = test['date'].map(dt.datetime.toordinal)
    if poly:
        polymonial = PolynomialFeatures(degree=11)
        poly_train = polymonial.fit_transform(pd.DataFrame(map_train_dates))
        poly_test = polymonial.transform(pd.DataFrame(map_test_dates))
        reg.fit(poly_train, train[column])
        y_pred = reg.predict(poly_test)
        sns.lineplot(x=train['date'], y=reg.predict(poly_train).squeeze(), label='Predict train', color='green', ax=ax)
    else:
        reg.fit(pd.DataFrame(map_train_dates), train[column])
        y_pred = reg.predict(pd.DataFrame(map_test_dates))
        sns.lineplot(x=train['date'], y=reg.predict(pd.DataFrame(map_train_dates)).squeeze(), label='Predict train', color='green', ax=ax)
    
    sns.scatterplot(x=train['date'], y=train[column].squeeze(), label='Actual train', s=15, ax=ax)
    sns.scatterplot(x=test['date'], y=test[column].squeeze(), label='Actual test', s=15, color='red', ax=ax)
    sns.lineplot(x=test['date'], y=y_pred.squeeze(), label='Predict test', color='green', ax=ax)
    
    metrics = dict(zip(['MSE', 'MAE', 'R2'], [mean_squared_error(test[column], y_pred), mean_absolute_error(test[column], y_pred), r2_score(test[column], y_pred)]))

    if ax is None:
        plt.legend()
        plt.xlabel('date')
        plt.ylabel(ylabel)
    else:
        ax.legend(loc='best')
        ax.set_xlabel('date')
        ax.set_ylabel(ylabel)
    return metrics


def draw_cool_plots(country_names, column, title, poly=False):
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(20, 12))
    fig.suptitle('Modele regresji liniowej', fontsize=20) if not poly else fig.suptitle('Modele regresji wielomianowej', fontsize=20)
    metrics = pd.DataFrame(columns=['MSE', 'MAE', 'R2'])
    for i, country_name in enumerate(country_names):
        x_cord, y_cord = divmod(i, 2)
        X_train, X_test = get_train_test(column, country_name, '2020-06-29', '2021-07-04', 5)
        reg_metrics = linear_regression(X_train, X_test, title, column, ax=ax[x_cord, y_cord], poly=poly)
        metrics = pd.concat([metrics, pd.DataFrame([reg_metrics], columns=reg_metrics.keys(), index=[country_name])])
        ax[x_cord, y_cord].set_title(country_name)
    display(metrics)

### 3.1. liczba nowych zachorowań, dla Ameryki Północnej

In [ ]:
draw_cool_plots(['Canada', 'Mexico', 'United States of America', 'Cuba'], 'daily_confirmed', 'Liczba nowych zachorowań')

### 3.2. liczba nowych śmierci, dla krajów skandynawskich

In [ ]:
draw_cool_plots(['Norway', 'Sweden', 'Finland', 'Denmark'], 'daily_deceased', 'Liczba nowych śmierci')

### 3.3. liczba nowych szczepień, dla państw Bliskiego Wschodu

In [ ]:
draw_cool_plots(['Israel', 'Turkey', 'Iran', 'Saudi Arabia'], 'daily_persons_vaccinated', 'Liczba nowych szczepień')

### 3.4 liczba nowych zachorowań mężczyzn, dla krajów bałkańskich

In [ ]:
draw_cool_plots(['Albania', 'Bosnia and Herzegovina', 'Croatia', 'Serbia'], 'daily_confirmed_male', 'Liczba nowych zachorowań meżczyzn')

### 3.4 liczba nowych śmierci mężczyzn, dla krajów bałkańskich

In [ ]:
draw_cool_plots(['Albania', 'Bosnia and Herzegovina', 'Croatia', 'Serbia'], 'daily_deceased_male', 'Liczba nowych śmierci mężczyzn')

## <center>Część 4</center>
### Użyj model regresji wielomianowej dla wszystkich 5 przypadków rozważanych w części 3 poziomu 3. Oceń, który model regresji (liniowy czy wielomianowy) okazał się "lepszy" dla każdego przypadku. W tym celu możesz użyć dowolnej miary dopasowania modelu. Uzasadnij swoje zdanie.

### 4.1. liczba nowych zachorowań, dla Ameryki Północnej

In [ ]:
draw_cool_plots(['Canada', 'Mexico', 'United States of America', 'Cuba'], 'daily_confirmed', 'Liczba nowych zachorowań', poly=True)

### 4.2. liczba nowych śmierci, dla krajów skandynawskich

In [ ]:
draw_cool_plots(['Norway', 'Sweden', 'Finland', 'Denmark'], 'daily_deceased', 'Liczba nowych śmierci', poly=True)

### 4.3. liczba nowych szczepień, dla krajów Bliskiego Wschodu

In [ ]:
draw_cool_plots(['Israel', 'Turkey', 'Iran', 'Saudi Arabia'], 'daily_persons_vaccinated', 'Liczba nowych szczepień', poly=True)

### 4.4 liczba nowych zachorowań mężczyzn, dla krajów bałkańskich

In [ ]:
draw_cool_plots(['Albania', 'Bosnia and Herzegovina', 'Croatia', 'Serbia'], 'daily_confirmed_male', 'Liczba nowych zachorowań meżczyzn', poly=True)

### 4.4 liczba śmierci zachorowań mężczyzn, dla krajów bałkańskich

In [ ]:
draw_cool_plots(['Albania', 'Bosnia and Herzegovina', 'Croatia', 'Serbia'], 'daily_deceased_male', 'Liczba nowych śmierci mężczyzn', poly=True)